# RAG Pipeline - Security Knowledge Base

Notebook này thực hiện RAG pipeline hoàn chỉnh với thứ tự đúng:
1. **LOAD** - Load documents (Sigma, MITRE ATT&CK, OWASP)
2. **SPLIT** - Text splitting
3. **EMBED** - Embedding và lưu vào ChromaDB
4. **RETRIEVE** - Query và Retrieve

## 📋 Thứ tự chạy các cells:

### Phase 1: Setup
- **Cell 1**: Imports
- **Cell 2**: Helper Functions

### Phase 2: LOAD Documents
- **Cell 3**: Load Sigma Rules
- **Cell 4**: Load MITRE ATT&CK
- **Cell 5**: Load OWASP Cheatsheets

### Phase 3: SPLIT Text
- **Cell 6**: Text Splitting và Combine All Documents
- **Cell 7**: Xem Chunks (Optional)

### Phase 4: EMBED
- **Cell 8**: Setup Embedding Model
- **Cell 9**: Embed và lưu vào ChromaDB

### Phase 5: RETRIEVE
- **Cell 10**: Basic Retrieval (Balanced Query)
- **Cell 11**: Hybrid Retrieval Setup (BM25 + Reranker)
- **Cell 12**: Hybrid Retrieval + Reranking

**Lưu ý:** Cần cài đặt:
- `pip install rank-bm25 flashrank sentence-transformers` cho hybrid retrieval và reranking


In [ ]:
# ===================================================================
# Phase 1: IMPORTS
# ===================================================================

from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_community.document_loaders import WebBaseLoader, AsyncChromiumLoader
from langchain_core.documents import Document
from langchain_chroma import Chroma
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain_text_splitters import RecursiveCharacterTextSplitter
try:
    from langchain_huggingface import HuggingFaceEmbeddings
except ImportError:
    from langchain_community.embeddings import HuggingFaceEmbeddings

# Reranker và Hybrid Retrieval
try:
    from rank_bm25 import BM25Okapi
except ImportError:
    print("⚠️ rank_bm25 chưa được cài đặt. Chạy: pip install rank-bm25")
    BM25Okapi = None

try:
    from sentence_transformers import CrossEncoder
    CROSS_ENCODER_AVAILABLE = True
except ImportError:
    CROSS_ENCODER_AVAILABLE = False
    print("⚠️ sentence-transformers chưa có CrossEncoder. Sẽ dùng simple reranking.")
    CrossEncoder = None

try:
    from flashrank import Ranker, RerankRequest
    FLASHRANK_AVAILABLE = True
except ImportError:
    FLASHRANK_AVAILABLE = False


import os
import yaml
import json
import requests
import glob
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time
import numpy as np
from typing import List, Tuple, Dict
from collections import Counter

print("✅ Imports completed")


USER_AGENT environment variable not set, consider setting it to identify your requests.


✅ Imports completed


In [ ]:
# ===================================================================
# Phase 1: HELPER FUNCTIONS
# ===================================================================

def parse_chroma_metadata(doc: Document) -> dict:
    """Parse metadata từ ChromaDB, convert JSON strings về lại list/dict"""
    metadata = doc.metadata.copy()
    json_fields = ['tags', 'references', 'detection', 'logsource', 'detection_keywords', 'falsepositives']
    
    for field in json_fields:
        if field in metadata and isinstance(metadata[field], str):
            try:
                metadata[field] = json.loads(metadata[field])
            except (json.JSONDecodeError, TypeError):
                pass
    return metadata

def format_doc_for_llm(doc: Document, include_full_rule: bool = False) -> str:
    """Format document với metadata để LLM dễ đọc"""
    metadata = parse_chroma_metadata(doc)
    output = []
    output.append(f"Title: {metadata.get('title', 'N/A')}")
    output.append(f"ID: {metadata.get('id', 'N/A')}")
    output.append(f"Status: {metadata.get('status', 'N/A')}")
    output.append(f"Level: {metadata.get('level', 'N/A')}")
    output.append(f"Description: {metadata.get('description', 'N/A')}")
    if metadata.get('author'):
        output.append(f"Author: {metadata.get('author')}")
    if metadata.get('tags'):
        tags = metadata['tags'] if isinstance(metadata['tags'], list) else []
        output.append(f"Tags: {', '.join(str(t) for t in tags)}")
    if metadata.get('logsource'):
        logsource = metadata['logsource'] if isinstance(metadata['logsource'], dict) else {}
        output.append(f"Log Source: {json.dumps(logsource, ensure_ascii=False)}")
    if metadata.get('detection'):
        detection = metadata['detection'] if isinstance(metadata['detection'], dict) else {}
        if 'keywords' in detection and detection['keywords']:
            keywords = detection['keywords']
            keywords_preview = keywords[:5] if len(keywords) > 5 else keywords
            output.append(f"Detection Keywords: {', '.join(str(k) for k in keywords_preview)}")
            if len(keywords) > 5:
                output.append(f"  (+ {len(keywords)-5} more keywords)")
        if 'condition' in detection:
            output.append(f"Detection Condition: {detection['condition']}")
    if metadata.get('references'):
        refs = metadata['references'] if isinstance(metadata['references'], list) else []
        if refs:
            output.append(f"References: {len(refs)} reference(s)")
            for ref in refs[:3]:
                output.append(f"  - {ref}")
            if len(refs) > 3:
                output.append(f"  ... và {len(refs)-3} reference(s) khác")
    output.append(f"\nContent:\n{doc.page_content}")
    if include_full_rule and metadata.get('full_rule'):
        output.append(f"\nFull Rule YAML:\n{metadata['full_rule']}")
    return "\n".join(output)

print("✅ Helper functions defined")


✅ Helper functions defined


## Phase 2: LOAD Documents

### 3.1. Load Sigma Rules


In [ ]:
# ===================================================================
# Phase 2: LOAD - Sigma Rules
# ===================================================================

sigma_path = r"D:\MCPLLM\test\sigma\rules\web\webserver_generic"

# Kiểm tra thư mục có tồn tại không
if not os.path.exists(sigma_path):
    print(f"Error: Thư mục không tồn tại: {sigma_path}")
    sigma_raw_docs = []
else:
    try:
        sigma_loader = DirectoryLoader(
            path=sigma_path,
            glob="**/*.yml",
            loader_cls=TextLoader,
            loader_kwargs={'encoding': 'utf-8'}
        )
        sigma_raw_docs = sigma_loader.load()
        print(f"✅ Đã load {len(sigma_raw_docs)} Sigma rule file(s)")
    except Exception as e:
        print(f"❌ Lỗi khi load: {e}")
        sigma_raw_docs = []

# Parse YAML và tạo processed docs
sigma_docs_processed = []

if sigma_raw_docs:
    print("\n📝 Xử lý và parse YAML...")
    for doc in sigma_raw_docs:
        try:
            parsed_yaml = yaml.safe_load(doc.page_content)
            if parsed_yaml:
                title = parsed_yaml.get('title', 'N/A')
                description = parsed_yaml.get('description', 'N/A')
                level = parsed_yaml.get('level', 'N/A')
                status = parsed_yaml.get('status', 'N/A')
                
                summary_content = f"Sigma Rule: {title}\nStatus: {status} | Level: {level}\nDescription: {description}"
                
                # Extract detection keywords
                detection = parsed_yaml.get('detection', {})
                keywords = detection.get('keywords', []) if isinstance(detection, dict) else []
                if keywords:
                    keywords_preview = keywords[:3] if len(keywords) > 3 else keywords
                    summary_content += f"\nKeywords: {', '.join(str(k) for k in keywords_preview)}"
                    if len(keywords) > 3:
                        summary_content += f" (+{len(keywords)-3} more)"
                
                new_doc = Document(
                    page_content=summary_content,
                    metadata={
                        "source": doc.metadata.get('source'),
                        "source_type": "sigma_rule",
                        "title": parsed_yaml.get('title'),
                        "id": parsed_yaml.get('id'),
                        "status": parsed_yaml.get('status'),
                        "level": parsed_yaml.get('level'),
                        "description": parsed_yaml.get('description'),
                        "author": parsed_yaml.get('author'),
                        "date": str(parsed_yaml.get('date', '')),
                        "modified": str(parsed_yaml.get('modified', '')),
                        "tags": parsed_yaml.get('tags', []),
                        "logsource": parsed_yaml.get('logsource', {}),
                        "detection": parsed_yaml.get('detection', {}),
                        "detection_keywords": keywords,
                        "detection_keywords_count": len(keywords),
                        "references": parsed_yaml.get('references', []),
                        "falsepositives": parsed_yaml.get('falsepositives', []),
                    }
                )
                sigma_docs_processed.append(new_doc)
        except Exception as e:
            print(f"Lỗi khi parse YAML: {e}")
    
    print(f"✅ Đã xử lý {len(sigma_docs_processed)} Sigma document(s)")
else:
    print("⚠️ Không có Sigma documents nào để xử lý.")


✅ Đã load 13 Sigma rule file(s)

📝 Xử lý và parse YAML...
✅ Đã xử lý 13 Sigma document(s)


In [ ]:
print(sigma_raw_docs[0])

page_content='title: F5 BIG-IP iControl Rest API Command Execution - Webserver
id: 85254a62-22be-4239-b79c-2ec17e566c37
related:
    - id: b59c98c6-95e8-4d65-93ee-f594dfb96b17
      type: similar
status: test
description: Detects POST requests to the F5 BIG-IP iControl Rest API "bash" endpoint, which allows the execution of commands on the BIG-IP
references:
    - https://f5-sdk.readthedocs.io/en/latest/apidoc/f5.bigip.tm.util.html#module-f5.bigip.tm.util.bash
    - https://community.f5.com/t5/technical-forum/icontrolrest-11-5-execute-bash-command/td-p/203029
    - https://community.f5.com/t5/technical-forum/running-bash-commands-via-rest-api/td-p/272516
author: Nasreddine Bencherchali (Nextron Systems), Thurein Oo
date: 2023-11-08
tags:
    - attack.execution
    - attack.t1190
    - attack.initial-access
logsource:
    category: webserver
detection:
    selection:
        cs-method: 'POST'
        cs-uri-query|endswith: '/mgmt/tm/util/bash'
    condition: selection
falsepositives:
  

### 3.2. Load MITRE ATT&CK Techniques


In [ ]:
# ===================================================================
# Phase 2: LOAD - MITRE ATT&CK Techniques
# ===================================================================

import requests
from bs4 import BeautifulSoup
import re
import time

def parse_mitre_techniques_from_enterprise_page(base_url="https://attack.mitre.org/techniques/enterprise/"):
    """
    Parse tất cả techniques từ trang enterprise của MITRE ATT&CK
    Format: T-ID\nName\nDescription
    """
    mitre_docs = []
    
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
        response = requests.get(base_url, headers=headers, timeout=30)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Tìm tất cả technique links
        technique_rows = soup.find_all('tr')
        
        for row in technique_rows:
            cells = row.find_all('td')
            if len(cells) >= 2:
                # Cell đầu tiên chứa ID và link
                first_cell = cells[0]
                link = first_cell.find('a', href=True)
                if link and '/techniques/T' in link['href']:
                    technique_id = link.get_text(strip=True)
                    # Cell thứ hai chứa tên
                    technique_name = cells[1].get_text(strip=True) if len(cells) > 1 else "Unknown"
                    
                    # Cell thứ ba có thể chứa description hoặc lấy từ tooltip
                    description = ""
                    if len(cells) > 2:
                        description = cells[2].get_text(strip=True)
                    
                    # Nếu không có description từ cell, lấy từ tooltip
                    if not description or len(description) < 20:
                        tooltip = first_cell.get('title') or first_cell.get('data-bs-original-title', '')
                        if tooltip:
                            description = tooltip
                    
                    # Format: ID\nName\nDescription
                    if technique_id and technique_name:
                        content = f"{technique_id}\n{technique_name}\n{description}" if description else f"{technique_id}\n{technique_name}"
                        
                        doc = Document(
                            page_content=content,
                            metadata={
                                'source_type': 'mitre_attack',
                                'source_url': urljoin(base_url, link['href']),
                                'technique_id': technique_id,
                                'technique_name': technique_name,
                                'description_length': len(description) if description else 0
                            }
                        )
                        mitre_docs.append(doc)
        
        print(f"✅ Đã parse {len(mitre_docs)} MITRE ATT&CK techniques từ trang enterprise")
        
    except Exception as e:
        print(f"❌ Lỗi khi parse MITRE techniques: {e}")
        import traceback
        traceback.print_exc()
    
    return mitre_docs

# Load MITRE techniques
print("📥 Đang load MITRE ATT&CK techniques...")
mitre_docs = parse_mitre_techniques_from_enterprise_page()

if mitre_docs:
    print(f"✅ Đã load {len(mitre_docs)} MITRE ATT&CK techniques")
else:
    print("⚠️ Không có MITRE documents nào.")


📥 Đang load MITRE ATT&CK techniques...
✅ Đã parse 216 MITRE ATT&CK techniques từ trang enterprise
✅ Đã load 216 MITRE ATT&CK techniques


In [ ]:
print(mitre_docs[0])

page_content='T1548
Abuse Elevation Control Mechanism
Adversaries may circumvent mechanisms designed to control elevate privileges to gain higher-level permissions. Most modern systems contain native elevation control mechanisms that are intended to limit privileges that a user can perform on a machine. Authorization has to be granted to specific users in order to perform tasks that can be considered of higher risk. An adversary can perform several methods to take advantage of built-in control mechanisms in order to escalate privileges on a system.' metadata={'source_type': 'mitre_attack', 'source_url': 'https://attack.mitre.org/techniques/T1548', 'technique_id': 'T1548', 'technique_name': 'Abuse Elevation Control Mechanism', 'description_length': 500}


In [ ]:
### 3.3. Load OWASP Cheatsheets

In [ ]:
# Load OWASP Cheatsheets từ thư mục local
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase

import os, glob
from langchain_community.document_loaders import DirectoryLoader, TextLoader

# Đường dẫn tới thư mục cheatsheets
cheatsheets_path = r"D:\MCPLLM\test\cheatsheets"

# Kiểm tra thư mục có tồn tại không
if not os.path.exists(cheatsheets_path):
    print(f"⚠️  Thư mục không tồn tại: {cheatsheets_path}")
    print(f"   Tạo thư mục hoặc kiểm tra đường dẫn")
    owasp_docs = []
else:
    # Dùng raw string để tránh lỗi escape
    folders = glob.glob(os.path.join(cheatsheets_path, "*"))
    
    # Thiết lập text loader kwargs
    # Thử dùng autodetect nếu có chardet, nếu không thì dùng utf-8
    try:
        import chardet
        text_loader_kwargs = {'autodetect_encoding': True}
    except ImportError:
        print("  ⚠️  Module 'chardet' chưa được cài đặt. Dùng encoding='utf-8' thay thế.")
        print("  💡 Để cài: pip install chardet")
        text_loader_kwargs = {'encoding': 'utf-8'}
    
    owasp_docs = []
    print(f"📂 Đang load OWASP cheatsheets từ: {cheatsheets_path}")
    
    for folder in folders:
        doc_type = os.path.basename(folder)
        
        try:
            if os.path.isdir(folder):
                loader = DirectoryLoader(
                    folder,
                    glob="**/*.md",  # ✅ luôn dùng dấu /
                    loader_cls=TextLoader,
                    loader_kwargs=text_loader_kwargs
                )
                folder_docs = loader.load()
                
            elif os.path.isfile(folder) and folder.endswith('.md'):
                loader = TextLoader(folder, **text_loader_kwargs)
                folder_docs = loader.load()
            else:
                continue  # Bỏ qua nếu không phải file .md
            
            for doc in folder_docs:
                doc.metadata["doc_type"] = doc_type
                doc.metadata["source_type"] = "owasp_cheatsheet"
                # Thêm source path vào metadata
                if 'source' not in doc.metadata:
                    doc.metadata["source"] = folder
                owasp_docs.append(doc)
            
            if folder_docs:
                print(f"  ✅ Loaded {len(folder_docs)} docs từ: {doc_type}")
        
        except Exception as e:
            print(f"  ⚠️  Lỗi khi load {folder}: {e}")
            continue
    
    print(f"\n✅ Total OWASP cheatsheet documents loaded: {len(owasp_docs)}")
    
    # Hiển thị một vài documents mẫu
    if owasp_docs:
        print(f"\n📋 Một vài documents mẫu:")
        for i, doc in enumerate(owasp_docs[:3]):
            doc_type = doc.metadata.get('doc_type', 'N/A')
            source = doc.metadata.get('source', 'N/A')
            if isinstance(source, str):
                source_name = os.path.basename(source)
            else:
                source_name = str(source)
            print(f"  [{i+1}] {doc_type} | {source_name} | {len(doc.page_content)} chars")


📂 Đang load OWASP cheatsheets từ: D:\MCPLLM\test\cheatsheets
  ✅ Loaded 1 docs từ: Abuse_Case_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: Access_Control_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: AJAX_Security_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: Attack_Surface_Analysis_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: Authentication_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: Authorization_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: Authorization_Testing_Automation_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: Automotive_Security.md
  ✅ Loaded 1 docs từ: Bean_Validation_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: Browser_Extension_Vulnerabilities_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: C-Based_Toolchain_Hardening_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: Choosing_and_Using_Security_Questions_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: CI_CD_Security_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: Clickjacking_Defense_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: Content_Security_Policy_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: Cookie_Theft_Mitigation_Cheat_Sheet.md
  ✅ Loaded 1 

In [ ]:
print(owasp_docs[0])

page_content='# Abuse Case Cheat Sheet (Historical)

## Archive Statement

Reviewers have identified that abuse cases are rarely used in practice. Additionally, the material is presented as a "getting started tutorial" which isn't appropriate for the cheat sheet series.

## Introduction

Often when the security level of an application is mentioned in requirements, the following _expressions_ are met:

- _The application must be secure_.
- _The application must defend against all attacks targeting this category of application_.
- _The application must defend against attacks from the OWASP TOP 10_
- ...

These security requirements are too generic, and thus useless for a development team...

In order to build a secure application, from a pragmatic point of view, it is important to identify the attacks which the application must defend against, according to its business and technical context. Abuse cases were a frequently recommended _threat modeling_ technique, and reviewing the [threat 

In [ ]:
# Text Splitting và Combine All Documents
# Tối ưu cho từng loại document: Sigma, MITRE, OWASP

# 1. Text splitter cho OWASP (Markdown files - có thể rất dài)
# OWASP cheatsheets thường là markdown, cần split để quản lý
owasp_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Phù hợp cho markdown content
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n## ", "\n\n### ", "\n\n", "\n", ". ", " ", ""]  # Ưu tiên split theo markdown headers
)

# 2. Text splitter cho MITRE (Format: T1548\nName\nDescription)
# Giữ nguyên technique nếu có thể, chỉ split nếu quá dài
mitre_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,  # Đủ lớn cho hầu hết techniques
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", ". "]  # Ưu tiên giữ nguyên format ID\nName\nDescription
)

# Combine tất cả documents
all_docs = []

# =================================================================
# 1. SIGMA RULES
# =================================================================
# Sigma rules đã được processed, format ngắn gọn, không cần split
if 'sigma_docs_processed' in locals() and sigma_docs_processed:
    for doc in sigma_docs_processed:
        # Đảm bảo metadata có source_type
        if 'source_type' not in doc.metadata:
            doc.metadata['source_type'] = 'sigma_rule'
        all_docs.append(doc)
    print(f"✅ Added {len(sigma_docs_processed)} Sigma rule documents (không split)")

# =================================================================
# 2. MITRE ATT&CK TECHNIQUES
# =================================================================
# Format: T1548\nName\nDescription
# Mỗi technique là một document riêng, chỉ split nếu description quá dài
if 'mitre_docs' in locals() and mitre_docs:
    mitre_split = []
    for doc in mitre_docs:
        # Đảm bảo metadata có source_type
        if 'source_type' not in doc.metadata:
            doc.metadata['source_type'] = 'mitre_attack'
        
        # Nếu description quá dài (>2000 chars), split theo paragraph
        if len(doc.page_content) > 2000:
            chunks = mitre_splitter.split_documents([doc])
            # Giữ metadata cho mỗi chunk
            for chunk in chunks:
                chunk.metadata['source_type'] = 'mitre_attack'
            mitre_split.extend(chunks)
        else:
            # Giữ nguyên document - mỗi technique là một chunk
            mitre_split.append(doc)
    
    all_docs.extend(mitre_split)
    long_docs = sum(1 for doc in mitre_docs if len(doc.page_content) > 2000)
    print(f"✅ Added {len(mitre_split)} MITRE ATT&CK documents (from {len(mitre_docs)} techniques)")
    if long_docs > 0:
        print(f"   - {long_docs} techniques đã được split do description quá dài")

# =================================================================
# 3. OWASP CHEATSHEETS
# =================================================================
# OWASP cheatsheets là markdown files, có thể rất dài, cần split
if 'owasp_docs' in locals() and owasp_docs:
    owasp_split = []
    for doc in owasp_docs:
        # Đảm bảo metadata có source_type
        if 'source_type' not in doc.metadata:
            doc.metadata['source_type'] = 'owasp_cheatsheet'
        
        # Split OWASP documents (markdown có thể rất dài)
        chunks = owasp_splitter.split_documents([doc])
        # Giữ metadata cho mỗi chunk
        for chunk in chunks:
            chunk.metadata['source_type'] = 'owasp_cheatsheet'
            # Giữ nguyên doc_type và source từ document gốc
            if 'doc_type' in doc.metadata:
                chunk.metadata['doc_type'] = doc.metadata['doc_type']
            if 'source' in doc.metadata:
                chunk.metadata['source'] = doc.metadata['source']
        owasp_split.extend(chunks)
    
    all_docs.extend(owasp_split)
    print(f"✅ Added {len(owasp_split)} OWASP cheatsheet document chunks (from {len(owasp_docs)} docs)")

# =================================================================
# TỔNG KẾT
# =================================================================
print(f"\n📊 Tổng cộng: {len(all_docs)} documents để embed và lưu vào ChromaDB")
print(f"\n💡 Chiến lược splitting:")
print(f"   - Sigma rules: Giữ nguyên (không split)")
print(f"   - MITRE techniques: Giữ nguyên, chỉ split nếu >2000 chars")
print(f"   - OWASP cheatsheets: Split theo markdown headers và paragraphs")


✅ Added 13 Sigma rule documents (không split)
✅ Added 216 MITRE ATT&CK documents (from 216 techniques)
✅ Added 3332 OWASP cheatsheet document chunks (from 107 docs)

📊 Tổng cộng: 3561 documents để embed và lưu vào ChromaDB

💡 Chiến lược splitting:
   - Sigma rules: Giữ nguyên (không split)
   - MITRE techniques: Giữ nguyên, chỉ split nếu >2000 chars
   - OWASP cheatsheets: Split theo markdown headers và paragraphs


In [ ]:
# Khởi tạo embedding model (dùng HuggingFace local)
print("📥 Đang tải embedding model...")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)
print("✅ Embedding model đã sẵn sàng")


📥 Đang tải embedding model...
✅ Embedding model đã sẵn sàng


In [ ]:
# Tạo đường dẫn lưu ChromaDB
persist_directory = r"D:\MCPLLM\test\chroma_sec_db"
os.makedirs(persist_directory, exist_ok=True)

# Chuẩn bị documents với metadata tương thích ChromaDB
print(f"💾 Đang tạo/load ChromaDB tại: {persist_directory}")

# Kiểm tra all_docs đã được tạo chưa
if 'all_docs' not in locals() or not all_docs:
    print("⚠️  Chưa có all_docs. Hãy chạy cell 'Text Splitting và Combine All Documents' trước.")
    print("    Đang dùng sigma_docs_processed tạm thời...")
    docs_to_process = sigma_docs_processed if 'sigma_docs_processed' in locals() and sigma_docs_processed else []
else:
    docs_to_process = all_docs

if not docs_to_process:
    print("❌ Không có documents nào để embed!")
    vectorstore = None
else:
    print(f"📝 Đang chuẩn bị {len(docs_to_process)} documents để embed...")
    
    # Convert list/dict trong metadata thành string cho ChromaDB
    docs_for_chroma = []
    for doc in docs_to_process:
        new_metadata = {}
        for key, value in doc.metadata.items():
            if isinstance(value, (list, dict)):
                # Convert list/dict thành JSON string
                new_metadata[key] = json.dumps(value, ensure_ascii=False)
            elif value is None:
                continue  # Skip None values
            else:
                new_metadata[key] = value
        
        # Tạo document mới với metadata đã xử lý
        new_doc = Document(
            page_content=doc.page_content,
            metadata=new_metadata
        )
        docs_for_chroma.append(new_doc)
    
    # Filter complex metadata một lần nữa để chắc chắn
    docs_for_chroma = filter_complex_metadata(docs_for_chroma)
    
    print(f"📊 Thống kê documents:")
    if 'all_docs' in locals() and all_docs:
        sigma_count = sum(1 for d in docs_for_chroma if d.metadata.get('source_type') == 'sigma_rule')
        mitre_count = sum(1 for d in docs_for_chroma if d.metadata.get('source_type') == 'mitre_attack')
        owasp_count = sum(1 for d in docs_for_chroma if d.metadata.get('source_type') == 'owasp_cheatsheet')
        print(f"   - Sigma rules: {sigma_count}")
        print(f"   - MITRE ATT&CK: {mitre_count}")
        print(f"   - OWASP cheatsheets: {owasp_count}")
    
    # Tạo ChromaDB vector store
    print(f"\n🔄 Đang embed và lưu vào ChromaDB...")
    vectorstore = Chroma.from_documents(
        documents=docs_for_chroma,
        embedding=embeddings,
        persist_directory=persist_directory,
        collection_name="security_knowledge_base"  # Tên collection mới cho tất cả
    )
    print(f"✅ Đã lưu {len(docs_for_chroma)} documents vào ChromaDB")


💾 Đang tạo/load ChromaDB tại: D:\MCPLLM\test\chroma_sec_db
📝 Đang chuẩn bị 3561 documents để embed...
📊 Thống kê documents:
   - Sigma rules: 13
   - MITRE ATT&CK: 216
   - OWASP cheatsheets: 3332

🔄 Đang embed và lưu vào ChromaDB...
✅ Đã lưu 3561 documents vào ChromaDB


In [ ]:
# Lấy ra bộ sưu tập vector từ vectorstore
collection = vectorstore._collection

# Lấy 1 embedding từ database
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]

# Kiểm tra số chiều (số phần tử trong vector)
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

NameError: name 'vectorstore' is not defined

In [ ]:
# ✅ Test query Chroma vectorstore (similarity / MMR / filter theo metadata)
import os, textwrap, json
from pprint import pprint

from langchain_community.vectorstores import Chroma

def ensure_vectorstore(persist_dir, embeddings, collection="security_knowledge_base"):
    # Nếu đã có vectorstore trong RAM thì dùng lại
    if 'vectorstore' in globals() and vectorstore is not None:
        return vectorstore
    # Ngược lại, mở từ ổ đĩa
    vs = Chroma(
        persist_directory=persist_dir,
        collection_name=collection,
        embedding_function=embeddings
    )
    return vs

def pretty(doc, score=None, idx=None, width=110, preview=400):
    h = []
    if idx is not None:
        h.append(f"[{idx}]")
    if score is not None:
        h.append(f"Score: {score:.4f}")
    title = doc.metadata.get("title") or doc.metadata.get("cheatsheet_name") or doc.metadata.get("id") or "(no title)"
    src_type = doc.metadata.get("source_type")
    h.append(f"Title: {title}")
    if src_type:
        h.append(f"SourceType: {src_type}")
    print(" | ".join(h))
    # nguồn
    src = doc.metadata.get("source_url") or doc.metadata.get("source")
    if src:
        print("Source:", src)
    # tag/technique/tags tóm tắt
    for key in ("tags","technique_id","technique_name","level","yaml_path"):
        if key in doc.metadata and doc.metadata[key]:
            val = doc.metadata[key]
            if isinstance(val, (list, dict)):
                val = json.dumps(val, ensure_ascii=False)
            print(f"{key}:", val)
    # preview nội dung
    content = (doc.page_content or "").strip().replace("\r", "")
    preview_text = content[:preview] + ("..." if len(content) > preview else "")
    print(textwrap.fill(preview_text, width=width))
    print("-" * width)

# ====== RUN ======
if 'persist_directory' not in globals():
    persist_directory = r"D:\MCPLLM\test\chroma_sec_db"

if 'embeddings' not in globals():
    raise RuntimeError("Chưa có biến `embeddings`. Hãy khởi tạo OpenAI/HF embeddings trước.")

vs = ensure_vectorstore(persist_directory, embeddings, collection="security_knowledge_base")
print("✅ Vectorstore ready.")
print("📦 Collection size (approx):", vs._collection.count())  # số vector (xấp xỉ)

# 👉 Đặt câu query bạn muốn test
query = "Detected SQL injection"
k = 5

print("\n🔎 Similarity search:")
hits = vs.similarity_search_with_score(query, k=k)
for i, (doc, score) in enumerate(hits, 1):
    pretty(doc, score=score, idx=i)

print("\n🧭 MMR (đa dạng hoá kết quả):")
mmr_docs = vs.max_marginal_relevance_search(query, k=min(6, k+1), fetch_k=20, lambda_mult=0.3)
for i, doc in enumerate(mmr_docs, 1):
    pretty(doc, score=None, idx=i)

print("\n🎯 Filter theo metadata (ví dụ: chỉ Sigma rules):")
try:
    # Chroma hỗ trợ 'where' filter trên metadata
    filtered = vs.similarity_search_with_score(
        query, k=k, filter={"source_type": "sigma_rule"}
    )
    if not filtered:
        print("  (Không có kết quả khớp filter {'source_type':'sigma_rule'})")
    else:
        for i, (doc, score) in enumerate(filtered, 1):
            pretty(doc, score=score, idx=i)
except TypeError:
    # Một số bản LangChain cũ dùng 'where' thay vì 'filter'
    filtered = vs.similarity_search_with_score(
        query, k=k, where={"source_type": "sigma_rule"}
    )
    for i, (doc, score) in enumerate(filtered, 1):
        pretty(doc, score=score, idx=i)

# (Tuỳ chọn) 🔁 Re-rank bằng FlashRank nếu có
try:
    from flashrank import Ranker, RerankRequest
    ranker = Ranker()  # mặc định dùng model nhẹ
    # Lấy top 10 từ similarity để rerank
    base_docs = vs.similarity_search(query, k=10)
    passages = [d.page_content[:1000] for d in base_docs]
    req = RerankRequest(query=query, passages=[{"id": str(i), "text": p} for i, p in enumerate(passages)])
    rr = ranker.rerank(req)
    print("\n⚡ FlashRank top-5 (reranked):")
    for r in rr[:5]:
        d = base_docs[int(r.document["id"])]
        pretty(d, score=r.relevance_score, idx=r.index)
except Exception as e:
    print("\n(ℹ️ Bỏ qua rerank FlashRank — chưa cài hoặc lỗi nhỏ:", e, ")")

print("\n✅ Done. Thay đổi biến `query` để thử thêm.")


C:\Users\Học Tập\AppData\Local\Temp\ipykernel_25616\3005272459.py:12: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vs = Chroma(


✅ Vectorstore ready.
📦 Collection size (approx): 3561

🔎 Similarity search:
[1] | Score: 0.6761 | Title: (no title) | SourceType: owasp_cheatsheet
Source: D:\MCPLLM\test\cheatsheets\Injection_Prevention_Cheat_Sheet.md
SQL Injection attacks can be divided into the following three classes:  - **Inband:**Â data is extracted using
the same channel that is used to inject the SQL code. This is the most straightforward kind of attack, in
which the retrieved data is presented directly in the application web page. - **Out-of-band:**Â data is
retrieved using a different channel (e.g., an email with the results of the que...
--------------------------------------------------------------------------------------------------------------
[2] | Score: 0.7689 | Title: (no title) | SourceType: owasp_cheatsheet
Source: D:\MCPLLM\test\cheatsheets\SQL_Injection_Prevention_Cheat_Sheet.md
## What Is a SQL Injection Attack?  Attackers can use SQL injection on an application if it has dynamic
database queries 

In [ ]:
# Hybrid search = Dense (Chroma) + Sparse (BM25) combiner
import sys, subprocess, math
from typing import List, Tuple
from rank_bm25 import BM25Okapi

# Nếu rank_bm25 chưa cài thì cài (cell sẽ lỗi nếu không có)
try:
    import rank_bm25  # noqa
except Exception:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "rank-bm25"])
    from rank_bm25 import BM25Okapi  # noqa

# Kiểm tra tiền điều kiện
if 'vs' not in globals() and 'vectorstore' not in globals():
    raise RuntimeError("Chưa có vectorstore (vs). Hãy load/khởi tạo Chroma trước.")

# prefer vs variable
_vs = vs if 'vs' in globals() else vectorstore

if 'docs_for_chroma' not in globals():
    raise RuntimeError("Cần biến `docs_for_chroma` (list of Documents) để xây BM25. Hãy tạo nó khi build Chroma.")

# Chuẩn bị corpus cho BM25 (đơn giản: page_content tokenized)
corpus_texts = [ (d.page_content or "").replace("\n"," ") for d in docs_for_chroma ]
# Tokenize: simple whitespace + lowercase; bạn có thể đưa tokenizer tốt hơn
tokenized_corpus = [ txt.lower().split() for txt in corpus_texts ]
bm25 = BM25Okapi(tokenized_corpus)

def normalize_scores(d: dict):
    """Min-max normalize in-place for dict key->score"""
    if not d:
        return {}
    vals = list(d.values())
    lo, hi = min(vals), max(vals)
    if lo == hi:
        # all equal -> give 1.0
        return {k: 1.0 for k in d}
    return {k: (v - lo) / (hi - lo) for k, v in d.items()}

def hybrid_search(
    query: str,
    k_dense: int = 10,
    k_sparse: int = 10,
    alpha: float = 0.6
) -> List[Tuple[object, float]]:
    """
    alpha: weight for dense score (0..1). final_score = alpha * dense_norm + (1-alpha) * sparse_norm
    returns list of (Document, combined_score) sorted desc
    """
    # 1) Dense (Chroma) top-k
    dense_hits = _vs.similarity_search_with_score(query, k=k_dense)
    # dense_hits: list of (Document, score). Depending on implementation score may be distance or similarity.
    # We'll assume larger score = more similar (as seen earlier). If you see scores like distances, invert them.
    dense_scores = {}
    for doc, s in dense_hits:
        key = id(doc)  # unique key
        dense_scores[key] = float(s)
    
    # 2) Sparse (BM25)
    qtok = query.lower().split()
    bm25_scores_arr = bm25.get_scores(qtok)  # length = len(corpus)
    sparse_scores = {}
    for idx, sc in enumerate(bm25_scores_arr):
        if sc <= 0:
            continue
        # key mapping must correspond to docs_for_chroma order
        key = id(docs_for_chroma[idx])
        sparse_scores[key] = float(sc)
    
    # 3) Normalize each score map
    dense_norm = normalize_scores(dense_scores)
    sparse_norm = normalize_scores(sparse_scores)
    
    # 4) Merge keys and compute combined score
    keys = set(dense_norm.keys()) | set(sparse_norm.keys())
    combined = {}
    for k in keys:
        dv = dense_norm.get(k, 0.0)
        sv = sparse_norm.get(k, 0.0)
        combined[k] = alpha * dv + (1 - alpha) * sv
    
    # 5) create list of (Document, score) and sort
    results = []
    # map id->doc for lookup
    id2doc = { id(docs_for_chroma[i]) : docs_for_chroma[i] for i in range(len(docs_for_chroma)) }
    for k, sc in sorted(combined.items(), key=lambda x: x[1], reverse=True):
        doc = id2doc.get(k)
        if doc:
            results.append((doc, sc))
    return results

# pretty printer (re-use earlier pretty if exists)
def pretty_result(doc, score, idx=None):
    title = doc.metadata.get("title") or doc.metadata.get("cheatsheet_name") or doc.metadata.get("id") or "(no title)"
    print(f"[{idx}] Score: {score:.4f} | Title: {title} | SourceType: {doc.metadata.get('source_type')}")
    print("  Preview:", (doc.page_content or "")[:300].replace("\n"," "))
    print("-" * 100)

# ===== Example usage =====
q = "Defense XSS Attack"
res = hybrid_search(q, k_dense=10, k_sparse=50, alpha=0.65)

print(f"Hybrid results for query: {q}\nTop {min(10,len(res))}:")
for i, (doc, sc) in enumerate(res[:10], 1):
    pretty_result(doc, sc, idx=i)

# Tweak alpha: alpha closer to 1 => favor dense; closer to 0 => favor BM25
print("\n(Điều chỉnh alpha để ưu tiên dense/sparse)")


Hybrid results for query: Defense XSS Attack
Top 10:
[1] Score: 0.3500 | Title: (no title) | SourceType: owasp_cheatsheet
  Preview: - Cookie Attributes - These change how JavaScript and browsers can interact with cookies. Cookie attributes try to limit the impact of an XSS attack but don’t prevent the execution of malicious content or address the root cause of the vulnerability. - Content Security Policy - An allowlist that prev
----------------------------------------------------------------------------------------------------
[2] Score: 0.2907 | Title: (no title) | SourceType: owasp_cheatsheet
  Preview: ### Summary  One final note: If deploying interceptors / filters as an XSS defense was a useful approach against XSS attacks, don't you think that it would be incorporated into all commercial Web Application Firewalls (WAFs) and be an approach that OWASP recommends in this cheat sheet?
--------------------------------------------------------------------------------------------------

In [ ]:
# --- Stable key builders (thay vì dùng id(doc)) ---
import hashlib, re
from rank_bm25 import BM25Okapi

def doc_key(d):
    """Khóa ổn định từ metadata + hash 256 ký tự đầu nội dung."""
    meta = d.metadata or {}
    src = meta.get("source_url") or meta.get("source") or ""
    title = meta.get("title") or meta.get("cheatsheet_name") or meta.get("id") or ""
    aux = meta.get("yaml_path") or meta.get("technique_id") or meta.get("technique_name") or ""
    head = (d.page_content or "")[:256]
    h = hashlib.md5(head.encode("utf-8","ignore")).hexdigest()[:8]
    return f"{src}|{title}|{aux}|{h}"

def payload_tokenize(text: str):
    text = (text or "").lower()
    return [t for t in re.findall(r"[a-z0-9_]+|[\$\{\}\|\&\;\=\.\:/\\'\"][\$\{\}\|\&\;\=\.\:/\\'\"0-9a-z_]*", text)
            if len(t) > 1 or t in ("'", '"', "/", "=", ".")]

def bm25_build_corpus(docs):
    corpus = []
    keys = []
    for d in docs:
        meta_bits = []
        for k in ("title","tags","yaml_path","technique_id","technique_name","cheatsheet_name"):
            v = d.metadata.get(k)
            if v is None: 
                continue
            if isinstance(v, list):
                v = " ".join(map(str, v))
            elif isinstance(v, dict):
                v = " ".join(f"{ik}:{iv}" for ik, iv in v.items())
            meta_bits.append(str(v))
        blob = " \n ".join([d.page_content or ""] + meta_bits)
        corpus.append(blob)
        keys.append(doc_key(d))
    return corpus, keys

# Build BM25 + key map một lần
corpus_texts, bm25_keys = bm25_build_corpus(docs_for_chroma)
bm25 = BM25Okapi([payload_tokenize(t) for t in corpus_texts])

# Map stable key -> Document gốc (từ docs_for_chroma)
key2doc = { doc_key(d): d for d in docs_for_chroma }

def _minmax(d):
    if not d: return {}
    vals = list(d.values()); lo, hi = min(vals), max(vals)
    if hi == lo: return {k: 1.0 for k in d}  # tất cả bằng nhau
    return {k: (v - lo) / (hi - lo) for k, v in d.items()}

def hybrid_search(query: str, k_dense=10, k_sparse=80, alpha=0.60):
    # 1) Dense từ Chroma (dùng stable key)
    dense_hits = vs.similarity_search_with_score(query, k=k_dense)
    # nếu thấy score là "khoảng cách" (nhỏ tốt), đảo dấu: s = -float(score)
    dense_scores = {}
    for d, s in dense_hits:
        k = doc_key(d)
        dense_scores[k] = float(s)

    # 2) Sparse từ BM25 (map theo bm25_keys)
    qtok = payload_tokenize(query)
    sparse_arr = bm25.get_scores(qtok)  # điểm cho toàn corpus
    sparse_scores = { bm25_keys[i]: float(sc) for i, sc in enumerate(sparse_arr) if sc > 0 }

    # 3) Chuẩn hoá & trộn
    dn, sn = _minmax(dense_scores), _minmax(sparse_scores)
    keys = set(dn) | set(sn)
    combo = {k: alpha*dn.get(k,0.0) + (1-alpha)*sn.get(k,0.0) for k in keys}

    # 4) Trả kết quả (Document, score) bằng key2doc
    results = []
    for k, sc in sorted(combo.items(), key=lambda x: x[1], reverse=True):
        d = key2doc.get(k)
        if d:
            results.append((d, sc))
    return results

def show_results(title, results, top=10, width=110, preview=360):
    import textwrap
    print(f"\n=== {title} (top {top}) ===")
    for i, (doc, sc) in enumerate(results[:top], 1):
        met = doc.metadata
        name = met.get("title") or met.get("cheatsheet_name") or met.get("id") or "(no title)"
        src = met.get("source_url") or met.get("source") or met.get("doc_type")
        print(f"[{i}] score={sc:.4f} | {name} | {met.get('source_type')}")
        if src: print("    src:", src)
        snippet = (doc.page_content or "").replace("\n"," ")[:preview]
        print("    ", textwrap.shorten(snippet, width=width))
    print("-"*width)

# Thử lại
query = "What is Rule Detection of SQL injection?"
dense_hits = vs.similarity_search_with_score(query, k=10)
show_results("Dense-only (Chroma)", dense_hits)

hyb_hits = hybrid_search(query, k_dense=10, k_sparse=80, alpha=0.60)
show_results("Hybrid (Dense+BM25, alpha=0.55)", hyb_hits)



=== Dense-only (Chroma) (top 10) ===
[1] score=0.6764 | (no title) | owasp_cheatsheet
    src: D:\MCPLLM\test\cheatsheets\Injection_Prevention_Cheat_Sheet.md
     ### Query languages The most famous form of injection is SQL Injection where an attacker can modify [...]
[2] score=0.6849 | (no title) | owasp_cheatsheet
    src: D:\MCPLLM\test\cheatsheets\Injection_Prevention_Cheat_Sheet.md
     SQL Injection attacks can be divided into the following three classes: - **Inband:**Â data is extracted [...]
[3] score=0.7527 | (no title) | owasp_cheatsheet
    src: D:\MCPLLM\test\cheatsheets\Injection_Prevention_Cheat_Sheet.md
     An SQL injection attack consists of insertion or "injection" of either a partial or complete SQL query [...]
[4] score=0.7770 | SQL Injection Strings In URI | sigma_rule
    src: D:\MCPLLM\test\sigma\rules\web\webserver_generic\web_sql_injection_in_access_logs.yml
     Sigma Rule: SQL Injection Strings In URI Status: test | Level: high Description: Detects potential

In [ ]:
# =============== LLM + RAG HYBRID cho LOG (LangChain + Groq) ===============
# - LLM: ChatGroq (GROQ_API_KEY từ biến môi trường)
# - VectorDB: Chroma (dense) + BM25 (sparse) = Hybrid
# - Pipeline:
#   (1) LLM sinh giả thuyết & câu hỏi bổ sung
#   (2) Tự query retriever (hybrid) theo từng câu hỏi
#   (3) LLM "Judge" kết luận attack + MITRE + confidence + nguồn

import os, sys, subprocess, json, re, hashlib, textwrap

# -------- 0) CÀI / IMPORT GÓI CẦN THIẾT --------
def _pip_install(pkgs):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + pkgs)
    except Exception as e:
        print("⚠️ pip install error:", e)

# Thư viện LangChain cần
try:
    from langchain_groq import ChatGroq
except Exception:
    _pip_install(["langchain-groq"])
    from langchain_groq import ChatGroq

try:
    from langchain_core.messages import SystemMessage, HumanMessage
    from langchain_core.runnables import RunnableLambda
except Exception:
    _pip_install(["langchain-core>=0.2.0"])
    from langchain_core.messages import SystemMessage, HumanMessage
    from langchain_core.runnables import RunnableLambda

try:
    from langchain_community.vectorstores import Chroma
except Exception:
    _pip_install(["langchain-community", "chromadb"])
    from langchain_community.vectorstores import Chroma

try:
    from langchain_huggingface import HuggingFaceEmbeddings
except Exception:
    _pip_install(["langchain-huggingface", "sentence-transformers"])
    from langchain_huggingface import HuggingFaceEmbeddings

# BM25 cho hybrid
try:
    from rank_bm25 import BM25Okapi
except Exception:
    _pip_install(["rank-bm25"])
    from rank_bm25 import BM25Okapi


# -------- 1) THIẾT LẬP LLM GROQ --------
# Đặt key vào biến môi trường trước khi chạy: os.environ["GROQ_API_KEY"] = "..."
if "GROQ_API_KEY" not in os.environ:
    print("⚠️  Thiếu GROQ_API_KEY trong biến môi trường. Đặt os.environ['GROQ_API_KEY']='<key>' trước khi gọi.")
# Model gợi ý: "llama-3.3-70b-versatile" hoặc "mixtral-8x7b-32768"
llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)

# -------- 2) MỞ / KHỞI TẠO VECTORSTORE & EMBEDDINGS --------
# Embedding MiniLM (như bạn dùng)
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    encode_kwargs={"normalize_embeddings": True}
)

# Thư mục Chroma đã persist từ cell trước của bạn
persist_directory = r"D:\MCPLLM\test\chroma_sec_db"

# Mở lại Chroma nếu có, ngược lại báo gợi ý
try:
    vs = Chroma(
        persist_directory=persist_directory,
        collection_name="security_knowledge_base",
        embedding_function=embeddings
    )
    print("✅ Chroma loaded:", persist_directory, "| Vectors ~", vs._collection.count())
except Exception as e:
    print("⚠️ Không mở được Chroma:", e)
    vs = None

# -------- 3) HYBRID RETRIEVER (Dense Chroma + Sparse BM25) --------
# Cần danh sách Document gốc để build BM25. Nếu bạn vẫn giữ biến docs_for_chroma thì dùng, còn không sẽ build sparse "lazy" từ DB.

def payload_tokenize(text: str):
    """Tokenizer giữ ký tự đặc biệt cho payload SOC."""
    text = (text or "").lower()
    tokens = re.findall(r"[a-z0-9_]+|[\$\{\}\|\&\;\=\.\:/\\'\"][\$\{\}\|\&\;\=\.\:/\\'\"0-9a-z_]*", text)
    return [t for t in tokens if len(t) > 1 or t in ("'", '"', "/", "=", ".")]

def doc_key(d):
    """Key ổn định cho mapping (metadata + hash)."""
    meta = getattr(d, "metadata", {}) or {}
    src = meta.get("source_url") or meta.get("source") or ""
    title = meta.get("title") or meta.get("cheatsheet_name") or meta.get("id") or ""
    aux = meta.get("yaml_path") or meta.get("technique_id") or meta.get("technique_name") or ""
    head = (getattr(d, "page_content", "") or "")[:256]
    h = hashlib.md5(head.encode("utf-8","ignore")).hexdigest()[:8]
    return f"{src}|{title}|{aux}|{h}"

def _minmax(d):
    if not d: return {}
    vals = list(d.values()); lo, hi = min(vals), max(vals)
    if hi == lo: return {k: 1.0 for k in d}
    return {k: (v - lo) / (hi - lo) for k, v in d.items()}

# Build BM25 corpus từ docs_for_chroma nếu có; nếu không, lấy N mẫu từ DB
if 'docs_for_chroma' in globals() and docs_for_chroma:
    # đã có docs_for_chroma từ pipeline embed
    sparse_docs = docs_for_chroma
else:
    # fallback: lấy ngẫu nhiên 2000 docs từ Chroma (nếu driver hỗ trợ). Nếu không, lấy bằng similarity với vài anchor query.
    sparse_docs = []
    if vs is not None:
        try:
            # Chroma không có API "load all docs" tiêu chuẩn qua LangChain;
            # dưới đây là cách khởi tạo sparse từ một số truy vấn anchor để có corpus đủ tốt.
            anchors = [
                "sql injection OR 1=1", "xss tag breaking", "command injection ; | &&",
                "F5 iControl bash endpoint", "MITRE T1003 LSASS", "path traversal ../../etc/passwd",
                "confluence cve-2022-26134", "java payload ${@java}", "waf rule xss", "authentication brute force"
            ]
            captured = {}
            for a in anchors:
                for d in vs.similarity_search(a, k=100):
                    k = doc_key(d)
                    if k not in captured:
                        captured[k] = d
            sparse_docs = list(captured.values())
            print(f"ℹ️ BM25 corpus from anchors: {len(sparse_docs)} docs")
        except Exception as e:
            print("⚠️ Không thể dựng corpus BM25 tự động:", e)
            sparse_docs = []

def bm25_build_corpus(docs):
    corpus, keys = [], []
    for d in docs:
        meta_bits = []
        for k in ("title","tags","yaml_path","technique_id","technique_name","cheatsheet_name","level"):
            v = (d.metadata or {}).get(k)
            if v is None: 
                continue
            if isinstance(v, list):
                v = " ".join(map(str, v))
            elif isinstance(v, dict):
                v = " ".join(f"{ik}:{iv}" for ik, iv in v.items())
            meta_bits.append(str(v))
        blob = " \n ".join([getattr(d, "page_content", "") or ""] + meta_bits)
        corpus.append(blob)
        keys.append(doc_key(d))
    return corpus, keys

bm25, bm25_keys, key2doc = None, [], {}
if sparse_docs:
    corpus_texts, bm25_keys = bm25_build_corpus(sparse_docs)
    bm25 = BM25Okapi([payload_tokenize(t) for t in corpus_texts])
    key2doc = { doc_key(d): d for d in sparse_docs }
    print("✅ BM25 ready on", len(sparse_docs), "docs")
else:
    print("⚠️ BM25 chưa sẵn sàng (thiếu corpus). Hybrid sẽ rơi về dense-only.")

def hybrid_search(query: str, k_dense=10, k_sparse=80, alpha=0.60):
    """Trả list[(Document, score)] theo điểm trộn hybrid. Nếu thiếu BM25, trả dense-only."""
    # Dense từ Chroma
    dense_hits = vs.similarity_search_with_score(query, k=k_dense) if vs is not None else []
    dense_scores = {}
    for d, s in dense_hits:
        # nếu score là khoảng cách (nhỏ tốt), có thể đảo dấu: s = -float(s)
        dense_scores[doc_key(d)] = float(s)

    if bm25 is None:
        # Dense-only
        return [(d, s) for d, s in dense_hits]

    # Sparse từ BM25
    qtok = payload_tokenize(query)
    sparse_arr = bm25.get_scores(qtok)
    sparse_scores = { bm25_keys[i]: float(sc) for i, sc in enumerate(sparse_arr) if sc > 0 }

    # Normalize & trộn
    dn, sn = _minmax(dense_scores), _minmax(sparse_scores)
    keys = set(dn) | set(sn)
    combo = {k: alpha*dn.get(k,0.0) + (1-alpha)*sn.get(k,0.0) for k in keys}

    # Map key -> doc
    results = []
    # ưu tiên lấy doc từ vs (dense) nếu có, sau đó từ key2doc (sparse)
    dense_map = { doc_key(d): d for d, _ in dense_hits }
    for k, sc in sorted(combo.items(), key=lambda x: x[1], reverse=True):
        d = dense_map.get(k) or key2doc.get(k)
        if d:
            results.append((d, sc))
    return results


# -------- 4) CHUỖI LLM: SINH CÂU HỎI & JUDGE KẾT LUẬN --------
SYSTEM_HYP = """Bạn là L2 SOC analyst. Nhận EVENT (log thô) và:
1) Sinh tối đa 5 giả thuyết kiểu tấn công có thể (vd: SQL Injection, XSS, LFI, Command Injection, Brute Force...)
2) Với mỗi giả thuyết, sinh 2–4 câu hỏi ngắn để truy KB nhằm tìm bằng chứng.
3) Trả JSON: {"hypotheses":[{"name":"...", "mitre":"Txxxx", "questions":["...", "..."]}, ...]}.
Chỉ trả JSON hợp lệ, không giải thích thêm.
"""

SYSTEM_JUDGE = """Bạn là bộ chấm điểm bằng chứng. Dựa trên CONTEXT (các đoạn KB đã truy hồi) và EVENT (log thô):
- Kết luận {"attack":"...", "mitre":"Txxxx", "confidence":0..1, "rationale":"...", "sources":[...]}
- Chỉ chọn attack nếu có bằng chứng rõ (pattern/rule/step khớp). Nếu không đủ bằng chứng, trả {"attack":"Unknown", "mitre":"", "confidence":0.0, ...}
Chỉ trả JSON hợp lệ.
"""

def llm_json(llm, system_prompt, user_content, max_retries=2):
    """Gọi LLM mong muốn JSON thuần."""
    for _ in range(max_retries+1):
        resp = llm.invoke([SystemMessage(content=system_prompt),
                           HumanMessage(content=user_content)])
        txt = (resp.content or "").strip()
        # Tìm block JSON
        try:
            # Nếu có text rác, bắt cặp {} đầu tiên
            start = txt.find("{")
            end = txt.rfind("}")
            if start >= 0 and end > start:
                return json.loads(txt[start:end+1])
        except Exception:
            continue
    raise ValueError("LLM không trả JSON hợp lệ:\n" + txt)

def retrieve_evidence_for_questions(questions, topk=5, alpha=0.60):
    """Hybrid retrieve cho list câu hỏi, trả danh sách đoạn bằng chứng (text + metadata)."""
    evid = []
    for q in questions:
        hits = hybrid_search(q, k_dense=topk, k_sparse=80, alpha=alpha)
        for d, sc in hits:
            evid.append({
                "text": d.page_content[:1500],
                "meta": d.metadata,
                "score": float(sc)
            })
    # lọc trùng theo source_url + yaml_path
    seen = set(); uniq = []
    for e in evid:
        m = e["meta"] or {}
        sig = (m.get("source_url") or m.get("source") or "") + "|" + str(m.get("yaml_path") or "")
        if sig in seen: 
            continue
        seen.add(sig)
        uniq.append(e)
    return uniq[:20]

def analyze_log(event_text: str, alpha=0.60):
    """Pipeline: sinh câu hỏi -> retrieve -> judge -> trả kết quả + evidence sử dụng."""
    # 1) Sinh giả thuyết + câu hỏi
    hyp_json = llm_json(llm, SYSTEM_HYP, f"EVENT:\n{event_text}")
    hypotheses = hyp_json.get("hypotheses", [])[:5]

    # 2) Với mỗi giả thuyết: retrieve bằng chứng
    hyp_results = []
    for hyp in hypotheses:
        qs = hyp.get("questions", [])[:4]
        evid = retrieve_evidence_for_questions(qs, topk=5, alpha=alpha)
        # Soạn context cho judge (rút gọn giữ nguồn)
        ctx_blocks = []
        for e in evid[:10]:
            meta = e["meta"] or {}
            src = meta.get("source_url") or meta.get("source") or ""
            head = (e["text"] or "")[:600]
            ctx_blocks.append(f"SOURCE: {src}\n{head}")
        ctx = "\n\n---\n\n".join(ctx_blocks)

        # 3) Judge kết luận
        judge_in = f"CONTEXT:\n{ctx}\n\nEVENT:\n{event_text}\n"
        judge_json = llm_json(llm, SYSTEM_JUDGE, judge_in)
        judge_json["hypothesis"] = hyp.get("name")
        judge_json["hyp_mitre"] = hyp.get("mitre","")
        judge_json["asked"] = qs
        # lưu top nguồn
        judge_json["top_sources"] = [ (e["meta"].get("source_url") or e["meta"].get("source") or "") for e in evid[:5] ]
        hyp_results.append(judge_json)

    # 4) Chọn kết quả tốt nhất theo confidence
    best = max(hyp_results, key=lambda x: x.get("confidence", 0.0), default={"attack":"Unknown","confidence":0.0})
    return {
        "event": event_text,
        "results": hyp_results,
        "best": best
    }

# ---------------- DEMO ----------------
demo_log = """What is SQL injection?"""

print("\n🔎 Demo analyze_log()...")
try:
    out = analyze_log(demo_log, alpha=0.60)  # alpha: ưu tiên dense ~60%, sparse ~40%
    print("\n=== KẾT LUẬN ===")
    print(json.dumps(out["best"], ensure_ascii=False, indent=2))
    print("\n=== CHI TIẾT (MỖI GIẢ THUYẾT) ===")
    for i, r in enumerate(out["results"], 1):
        print(f"\n[{i}] hyp={r.get('hypothesis')} -> attack={r.get('attack')} | conf={r.get('confidence')}")
        print("  asked:", r.get("asked"))
        print("  sources:", r.get("top_sources")[:3])
except Exception as e:
    print("⚠️ Demo lỗi nhỏ:", e)
    print("Kiểm tra: GROQ_API_KEY, Chroma persist_directory, và dữ liệu đã embed.")


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


✅ Chroma loaded: D:\MCPLLM\test\chroma_sec_db | Vectors ~ 3561
ℹ️ BM25 corpus from anchors: 766 docs
✅ BM25 ready on 766 docs

🔎 Demo analyze_log()...


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"



=== KẾT LUẬN ===
{
  "attack": "Unknown",
  "mitre": "",
  "confidence": 0.0,
  "rationale": "Không có bằng chứng rõ ràng về tấn công",
  "sources": [],
  "hypothesis": "SQL Injection",
  "hyp_mitre": "T1190",
  "asked": [
    "Có yêu cầu HTTP nào chứa ký tự đặc biệt như ';', '--', hoặc 'UNION' không?",
    "Liệu có thể xảy ra lỗi cơ sở dữ liệu khi nhập dữ liệu không?",
    "Có dấu hiệu của việc thực thi lệnh SQL không mong muốn không?"
  ],
  "top_sources": [
    "D:\\MCPLLM\\test\\cheatsheets\\Session_Management_Cheat_Sheet.md",
    "D:\\MCPLLM\\test\\cheatsheets\\REST_Assessment_Cheat_Sheet.md",
    "D:\\MCPLLM\\test\\cheatsheets\\OS_Command_Injection_Defense_Cheat_Sheet.md",
    "D:\\MCPLLM\\test\\cheatsheets\\Cross_Site_Scripting_Prevention_Cheat_Sheet.md",
    "D:\\MCPLLM\\test\\cheatsheets\\Injection_Prevention_Cheat_Sheet.md"
  ]
}

=== CHI TIẾT (MỖI GIẢ THUYẾT) ===

[1] hyp=SQL Injection -> attack=Unknown | conf=0.0
  asked: ["Có yêu cầu HTTP nào chứa ký tự đặc biệt như ';', 

In [ ]:
# ================= FULL CELL: RAG-HYBRID QA + Log-Analysis (LangChain + Groq) =================
# Copy/paste toàn bộ cell này vào Jupyter. Chỉnh GROQ_API_KEY và persist_directory trước khi chạy.
import os, sys, subprocess, json, re, hashlib, textwrap, random, time

# --------------------- helper: pip install khi cần ---------------------
def _pip_install(pkgs):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + pkgs)
    except Exception as e:
        print("⚠️ pip install error:", e)

# Các import chính (với fallback cài đặt)
try:
    from langchain_groq import ChatGroq
except Exception:
    _pip_install(["langchain-groq"])
    from langchain_groq import ChatGroq

try:
    from langchain_core.messages import SystemMessage, HumanMessage
    from langchain_core.runnables import RunnableLambda
except Exception:
    _pip_install(["langchain-core>=0.2.0"])
    from langchain_core.messages import SystemMessage, HumanMessage
    from langchain_core.runnables import RunnableLambda

try:
    from langchain_community.vectorstores import Chroma
except Exception:
    _pip_install(["langchain-community", "chromadb"])
    from langchain_community.vectorstores import Chroma

try:
    from langchain_huggingface import HuggingFaceEmbeddings
except Exception:
    _pip_install(["langchain-huggingface", "sentence-transformers"])
    from langchain_huggingface import HuggingFaceEmbeddings

try:
    from rank_bm25 import BM25Okapi
except Exception:
    _pip_install(["rank-bm25"])
    from rank_bm25 import BM25Okapi

# --------------------- Cấu hình: GROQ key & Chroma path ---------------------
# Đặt biến môi trường trước khi chạy (hoặc gán trực tiếp ở đây)
# Ví dụ: os.environ["GROQ_API_KEY"] = "sk-...."
if "GROQ_API_KEY" not in os.environ:
    print("⚠️ Thiếu GROQ_API_KEY trong biến môi trường. Hãy đặt: os.environ['GROQ_API_KEY']='<key>' trước khi chạy.")
# Chroma persist directory (chỉnh theo môi trường của bạn)
persist_directory = r"D:\MCPLLM\test\chroma_sec_db"  # <-- chỉnh đường dẫn này

# --------------------- Khởi tạo LLM ---------------------
# Model gợi ý: "llama-3.3-70b-versatile" hoặc adapt theo key/model bạn có
try:
    llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)
    print("✅ ChatGroq client initialized.")
except Exception as e:
    print("⚠️ Không khởi tạo được ChatGroq:", e)
    llm = None

# --------------------- Embeddings + Chroma ---------------------
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    encode_kwargs={"normalize_embeddings": True}
)

vs = None
try:
    vs = Chroma(
        persist_directory=persist_directory,
        collection_name="security_knowledge_base",
        embedding_function=embeddings
    )
    # Tries to show approximate count if API exposes it
    try:
        count = vs._collection.count()
    except Exception:
        count = "unknown"
    print("✅ Chroma loaded:", persist_directory, "| vectors:", count)
except Exception as e:
    print("⚠️ Không mở được Chroma:", e)
    vs = None

# --------------------- BM25 corpus builder & tokenizer ---------------------
def payload_tokenize(text: str):
    text = (text or "").lower()
    tokens = re.findall(r"[a-z0-9_]+|[\$\{\}\|\&\;\=\.\:/\\'\"][\$\{\}\|\&\;\=\.\:/\\'\"0-9a-z_]*", text)
    return [t for t in tokens if len(t) > 1 or t in ("'", '"', "/", "=", ".")]

def doc_key(d):
    meta = getattr(d, "metadata", {}) or {}
    src = meta.get("source_url") or meta.get("source") or ""
    title = meta.get("title") or meta.get("cheatsheet_name") or meta.get("id") or ""
    aux = meta.get("yaml_path") or meta.get("technique_id") or meta.get("technique_name") or ""
    head = (getattr(d, "page_content", "") or "")[:256]
    h = hashlib.md5(head.encode("utf-8","ignore")).hexdigest()[:8]
    return f"{src}|{title}|{aux}|{h}"

def _minmax(d):
    if not d: return {}
    vals = list(d.values()); lo, hi = min(vals), max(vals)
    if hi == lo: return {k: 1.0 for k in d}
    return {k: (v - lo) / (hi - lo) for k, v in d.items()}

# Build sparse_docs from Chroma anchors if docs_for_chroma missing
sparse_docs = []
if 'docs_for_chroma' in globals() and docs_for_chroma:
    sparse_docs = docs_for_chroma
else:
    if vs is not None:
        try:
            anchors = [
                "sql injection OR 1=1", "xss tag breaking", "command injection ; | &&",
                "F5 iControl bash endpoint", "MITRE T1003 LSASS", "path traversal ../../etc/passwd",
                "confluence cve-2022-26134", "java payload ${@java}", "waf rule xss", "authentication brute force",
                "sigma rule example", "mitre t1059", "cve exploit example"
            ]
            captured = {}
            for a in anchors:
                try:
                    hits = vs.similarity_search(a, k=100)
                except Exception:
                    hits = []
                for d in hits:
                    k = doc_key(d)
                    if k not in captured:
                        captured[k] = d
            sparse_docs = list(captured.values())
            print(f"ℹ️ BM25 corpus from anchors: {len(sparse_docs)} docs")
        except Exception as e:
            print("⚠️ Không thể dựng corpus BM25 tự động:", e)
            sparse_docs = []
    else:
        print("⚠️ Chroma không sẵn sàng, sẽ chạy dense-only.")

def bm25_build_corpus(docs):
    corpus, keys = [], []
    for d in docs:
        meta_bits = []
        for k in ("title","tags","yaml_path","technique_id","technique_name","cheatsheet_name","level"):
            v = (d.metadata or {}).get(k)
            if v is None:
                continue
            if isinstance(v, list):
                v = " ".join(map(str, v))
            elif isinstance(v, dict):
                v = " ".join(f"{ik}:{iv}" for ik, iv in v.items())
            meta_bits.append(str(v))
        blob = " \n ".join([getattr(d, "page_content", "") or ""] + meta_bits)
        corpus.append(blob)
        keys.append(doc_key(d))
    return corpus, keys

bm25, bm25_keys, key2doc = None, [], {}
if sparse_docs:
    corpus_texts, bm25_keys = bm25_build_corpus(sparse_docs)
    bm25 = BM25Okapi([payload_tokenize(t) for t in corpus_texts])
    key2doc = { doc_key(d): d for d in sparse_docs }
    print("✅ BM25 ready on", len(sparse_docs), "docs")
else:
    print("⚠️ BM25 chưa sẵn sàng (thiếu corpus). Hybrid sẽ rơi về dense-only.")

# --------------------- Hybrid search ---------------------
def hybrid_search(query: str, k_dense=10, k_sparse=80, alpha=0.60):
    dense_hits = []
    if vs is not None:
        try:
            dense_hits = vs.similarity_search_with_score(query, k=k_dense)
        except Exception as e:
            # fallback: empty
            dense_hits = []
    dense_scores = {}
    for d, s in dense_hits:
        # assume higher score = more relevant; adapt if driver gives distance
        dense_scores[doc_key(d)] = float(s)

    if bm25 is None:
        return [(d, s) for d, s in dense_hits]

    qtok = payload_tokenize(query)
    sparse_arr = bm25.get_scores(qtok)
    sparse_scores = { bm25_keys[i]: float(sc) for i, sc in enumerate(sparse_arr) if sc > 0 }

    dn, sn = _minmax(dense_scores), _minmax(sparse_scores)
    keys = set(dn) | set(sn)
    combo = {k: alpha*dn.get(k,0.0) + (1-alpha)*sn.get(k,0.0) for k in keys}

    results = []
    dense_map = { doc_key(d): d for d, _ in dense_hits }
    for k, sc in sorted(combo.items(), key=lambda x: x[1], reverse=True):
        d = dense_map.get(k) or key2doc.get(k)
        if d:
            results.append((d, sc))
    return results

# --------------------- LLM JSON helpers (giữ nguyên) ---------------------
SYSTEM_HYP = """Bạn là L2 SOC analyst. Nhận EVENT (log thô) và:
1) Sinh tối đa 5 giả thuyết kiểu tấn công có thể (vd: SQL Injection, XSS, LFI, Command Injection, Brute Force...)
2) Với mỗi giả thuyết, sinh 2–4 câu hỏi ngắn để truy KB nhằm tìm bằng chứng.
3) Trả JSON: {"hypotheses":[{"name":"...", "mitre":"Txxxx", "questions":["...", "..."]}, ...]}.
Chỉ trả JSON hợp lệ, không giải thích thêm.
"""

SYSTEM_JUDGE = """Bạn là bộ chấm điểm bằng chứng. Dựa trên CONTEXT (các đoạn KB đã truy hồi) và EVENT (log thô):
- Kết luận {"attack":"...", "mitre":"Txxxx", "confidence":0..1, "rationale":"...", "sources":[...]}.
- Chỉ chọn attack nếu có bằng chứng rõ (pattern/rule/step khớp). Nếu không đủ bằng chứng, trả {"attack":"Unknown", "mitre":"", "confidence":0.0, ...}
Chỉ trả JSON hợp lệ.
"""

def llm_json(llm, system_prompt, user_content, max_retries=2):
    if llm is None:
        raise ValueError("LLM chưa khởi tạo.")
    for _ in range(max_retries+1):
        resp = llm.invoke([SystemMessage(content=system_prompt),
                           HumanMessage(content=user_content)])
        txt = (resp.content or "").strip()
        try:
            start = txt.find("{")
            end = txt.rfind("}")
            if start >= 0 and end > start:
                return json.loads(txt[start:end+1])
        except Exception:
            continue
    raise ValueError("LLM không trả JSON hợp lệ:\n" + txt)

def retrieve_evidence_for_questions(questions, topk=5, alpha=0.60):
    evid = []
    for q in questions:
        hits = hybrid_search(q, k_dense=topk, k_sparse=80, alpha=alpha)
        for d, sc in hits:
            evid.append({
                "text": d.page_content[:1500],
                "meta": d.metadata,
                "score": float(sc)
            })
    seen = set(); uniq = []
    for e in evid:
        m = e["meta"] or {}
        sig = (m.get("source_url") or m.get("source") or "") + "|" + str(m.get("yaml_path") or "")
        if sig in seen:
            continue
        seen.add(sig)
        uniq.append(e)
    return uniq[:20]

def analyze_log(event_text: str, alpha=0.60):
    hyp_json = llm_json(llm, SYSTEM_HYP, f"EVENT:\n{event_text}")
    hypotheses = hyp_json.get("hypotheses", [])[:5]
    hyp_results = []
    for hyp in hypotheses:
        qs = hyp.get("questions", [])[:4]
        evid = retrieve_evidence_for_questions(qs, topk=5, alpha=alpha)
        ctx_blocks = []
        for e in evid[:10]:
            meta = e["meta"] or {}
            src = meta.get("source_url") or meta.get("source") or ""
            head = (e["text"] or "")[:600]
            ctx_blocks.append(f"SOURCE: {src}\n{head}")
        ctx = "\n\n---\n\n".join(ctx_blocks)
        judge_in = f"CONTEXT:\n{ctx}\n\nEVENT:\n{event_text}\n"
        judge_json = llm_json(llm, SYSTEM_JUDGE, judge_in)
        judge_json["hypothesis"] = hyp.get("name")
        judge_json["hyp_mitre"] = hyp.get("mitre","")
        judge_json["asked"] = qs
        judge_json["top_sources"] = [ (e["meta"].get("source_url") or e["meta"].get("source") or "") for e in evid[:5] ]
        hyp_results.append(judge_json)
    best = max(hyp_results, key=lambda x: x.get("confidence", 0.0), default={"attack":"Unknown","confidence":0.0})
    return {
        "event": event_text,
        "results": hyp_results,
        "best": best
    }

# --------------------- RAG-QA prompts & function ---------------------
SYSTEM_PLAN = """Bạn là trợ lý an toàn thông tin. Nhận CÂU HỎI của người dùng.
Hãy:
1) Phân rã tối đa 5 câu hỏi con cần tra cứu KB để trả lời.
2) Trả JSON hợp lệ:
{"subquestions": ["...", "...", "..."]}

Chỉ trả JSON, không giải thích thêm.
"""

SYSTEM_WRITE = """Bạn là trợ lý an toàn thông tin trả lời NGẮN GỌN, CHÍNH XÁC và có dẫn nguồn.
Dựa trên CONTEXT (các đoạn KB đã truy hồi) và QUESTION (câu hỏi người dùng), hãy:
- Trả lời trực tiếp, súc tích. Nếu có cảnh báo/ngoại lệ, nêu rõ.
- Khi có thể, nêu thêm MITRE ID liên quan hoặc CVE nếu câu hỏi dính tới lỗ hổng/chiến thuật.
- Cuối câu trả lời, thêm mục 'Nguồn:' dạng danh sách ngắn gọn (tối đa 5 dòng) gồm tiêu đề/ngắn gọn + URL.

Chỉ trả nội dung Markdown (đừng trả JSON).
"""

def qa_answer(user_question: str, alpha=0.60, topk=5):
    # 1) Lập kế hoạch (subquestions)
    try:
        plan = llm_json(llm, SYSTEM_PLAN, f"QUESTION:\n{user_question}")
        subqs = plan.get("subquestions", [])[:5]
    except Exception:
        subqs = [user_question]
    if not subqs:
        subqs = [user_question]

    # 2) Retrieve
    evid = retrieve_evidence_for_questions(subqs, topk=topk, alpha=alpha)

    # 3) Build context
    ctx_blocks, source_urls = [], []
    for e in evid[:12]:
        meta = e.get("meta") or {}
        url = meta.get("source_url") or meta.get("source") or ""
        title = meta.get("title") or meta.get("technique_id") or meta.get("cheatsheet_name") or ""
        head = (e.get("text") or "")[:800]
        ctx_blocks.append(f"SOURCE: {title or url}\nURL: {url}\n{head}")
        if url:
            source_urls.append(url)
    context = "\n\n---\n\n".join(ctx_blocks)

    # 4) Write final answer
    writer_in = f"CONTEXT:\n{context}\n\nQUESTION:\n{user_question}\n"
    if llm is None:
        raise ValueError("LLM chưa khởi tạo.")
    resp = llm.invoke([SystemMessage(content=SYSTEM_WRITE),
                       HumanMessage(content=writer_in)])
    answer_md = (resp.content or "").strip()

    # Compact sources
    uniq = []
    seen = set()
    for u in source_urls:
        if u and u not in seen:
            uniq.append(u); seen.add(u)
        if len(uniq) >= 5:
            break

    return {
        "question": user_question,
        "answer_markdown": answer_md,
        "sources": uniq,
        "subquestions": subqs
    }

# --------------------- Simple router (QA vs log analysis) ---------------------
def smart_router(user_input: str) -> str:
    s = (user_input or "").lower()
    payloadish = bool(re.search(r"(\.\./|%[0-9a-f]{2}|;|\||&&|\$\{|\bselect\b|\binsert\b|\bunion\b)", s))
    longish = len(s) > 300
    if payloadish or longish:
        return "log"
    return "qa"

# --------------------- Demo: 2 ví dụ (QA + Log) ---------------------
if __name__ == "__main__":
    # Demo QA
    try:
        q = "So sánh XSS Reflected với Stored, cách phát hiện và rule Sigma mẫu?"
        print("\n🔎 RAG-QA DEMO")
        qa = qa_answer(q, alpha=0.55, topk=6)
        print("\n--- QUESTION ---\n", q)
        print("\n--- ANSWER (markdown) ---\n", qa["answer_markdown"])
        print("\n--- SUBQUESTIONS ---\n", qa["subquestions"])
        print("\n--- SOURCES ---\n", qa["sources"])
    except Exception as e:
        print("⚠️ QA demo lỗi:", e)

    # Demo Log analysis (sử dụng pipeline cũ)
    try:
        demo_log = "GET /../../etc.conf HTTP/1.1\nHost: victim\nUser-Agent: test\n"
        print("\n\n🔎 LOG-ANALYZE DEMO")
        out = analyze_log(demo_log, alpha=0.60)
        print("\n=== BEST ===\n", json.dumps(out["best"], ensure_ascii=False, indent=2))
        print("\n=== ALL HYPOTHESES ===")
        for r in out["results"]:
            print("-", r.get("hypothesis"), "->", r.get("attack"), "| conf=", r.get("confidence"))
    except Exception as e:
        print("⚠️ Log analyze demo lỗi:", e)

# ================================================================================================


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


✅ ChatGroq client initialized.
✅ Chroma loaded: D:\MCPLLM\test\chroma_sec_db | vectors: 3561
ℹ️ BM25 corpus from anchors: 923 docs
✅ BM25 ready on 923 docs

🔎 RAG-QA DEMO


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"



--- QUESTION ---
 So sánh XSS Reflected với Stored, cách phát hiện và rule Sigma mẫu?

--- ANSWER (markdown) ---
 XSS Reflected và Stored là hai loại tấn công Cross-Site Scripting (XSS) khác nhau:

- **XSS Reflected**: Là loại tấn công xảy ra khi ứng dụng web nhận dữ liệu từ người dùng và phản ánh lại trên trang web mà không kiểm tra hoặc lọc dữ liệu. Điều này cho phép kẻ tấn công tiêm mã độc vào trang web thông qua các yêu cầu HTTP. Ví dụ, nếu một trang web có một biểu mẫu tìm kiếm và không kiểm tra dữ liệu nhập vào, kẻ tấn công có thể tiêm mã độc vào trường tìm kiếm và khi người dùng nhấp vào liên kết, mã độc sẽ được thực thi.

- **XSS Stored**: Là loại tấn công xảy ra khi dữ liệu từ người dùng được lưu trữ trên máy chủ và sau đó được hiển thị trên trang web mà không được kiểm tra hoặc lọc. Điều này cho phép kẻ tấn công lưu trữ mã độc trên máy chủ và khi người dùng truy cập trang web, mã độc sẽ được thực thi.

Để phát hiện các tấn công XSS, có thể sử dụng các phương pháp sau:

- **K

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"



=== BEST ===
 {
  "attack": "Path Traversal",
  "mitre": "T1083",
  "confidence": 0.8,
  "rationale": "Yêu cầu GET đến đường dẫn /../../etc.conf cho thấy một nỗ lực để truy cập file hệ thống bằng cách sử dụng traversal directory, liên quan đến khám phá và truy cập file trên hệ thống mục tiêu.",
  "sources": [
    "https://attack.mitre.org/techniques/T1083"
  ],
  "hypothesis": "Path Traversal",
  "hyp_mitre": "T1005",
  "asked": [
    "Có dấu hiệu truy cập file hệ thống không?",
    "Liệu yêu cầu GET có thể truy cập được file ngoài thư mục gốc không?",
    "Có lỗi nào liên quan đến việc kiểm soát đường dẫn không?"
  ],
  "top_sources": [
    "D:\\MCPLLM\\test\\cheatsheets\\DotNet_Security_Cheat_Sheet.md",
    "D:\\MCPLLM\\test\\cheatsheets\\Denial_of_Service_Cheat_Sheet.md",
    "D:\\MCPLLM\\test\\cheatsheets\\Input_Validation_Cheat_Sheet.md",
    "D:\\MCPLLM\\test\\cheatsheets\\Password_Storage_Cheat_Sheet.md",
    "https://attack.mitre.org/techniques/T1222"
  ]
}

=== ALL HYPOTHESES

In [ ]:
# --------------------- Generate Splunk Rule (Saved Search) from question/log ---------------------
SYSTEM_SPLUNK_RULE = """Bạn là SOC engineer + Splunk expert.
Nhiệm vụ: dựa trên CONTEXT (đoạn KB đã truy hồi) và INPUT (có thể là một EVENT log hoặc câu hỏi kỹ thuật),
hãy sinh một Splunk savedsearch / detection rule gồm:
- title: ngắn gọn
- description: 1-2 câu (gồm rationale & điều kiện phát hiện)
- spl: câu lệnh SPL (đã test logic, không dài quá 5 dòng; dùng rex, search, stats, transaction, where, eval, mvexpand khi cần)
- condition_examples: 2 ví dụ payload/log mẫu khớp rule
- severity: one of ["low","medium","high","critical"]
- mitre: list of MITRE ATT&CK ids (nếu có)
- schedule: cron-like hoặc interval ("*/5 * * * *" or "every 5m")
- alert_actions: list (ví dụ: ["email","notable","webhook","runbook"])
- tags: short list of tags (e.g., ["web","sqli","input-validation"])
- detection_notes: ngắn gọn các false-positive possible & tuning suggestions

Trả về **một JSON** hợp lệ với những field trên. Chỉ trả JSON, KHÔNG giải thích thêm.
"""

def generate_splunk_rule(input_text: str, alpha=0.60, topk=6):
    """
    Pipeline:
      1) tạo sub-queries (dùng llm_json như PLAN) -> retrieve evidence
      2) build context -> gọi llm_json với SYSTEM_SPLUNK_RULE -> parse JSON -> trả về dict
    """
    # 1) Sử dụng plan để lấy subquestions (tái dùng SYSTEM_PLAN nếu muốn)
    try:
        plan = llm_json(llm, SYSTEM_PLAN, f"QUESTION (for rule generation):\n{input_text}")
        subqs = plan.get("subquestions", [])[:6]
    except Exception:
        subqs = [input_text]

    # 2) Retrieve evidence
    evid = retrieve_evidence_for_questions(subqs, topk=topk, alpha=alpha)
    ctx_blocks = []
    for e in evid[:12]:
        meta = e.get("meta") or {}
        url = meta.get("source_url") or meta.get("source") or ""
        title = meta.get("title") or meta.get("cheatsheet_name") or ""
        head = (e.get("text") or "")[:800]
        ctx_blocks.append(f"SOURCE: {title or url}\nURL: {url}\n{head}")
    context = "\n\n---\n\n".join(ctx_blocks)

    # 3) Ask LLM to generate Splunk rule JSON
    llm_input = f"CONTEXT:\n{context}\n\nINPUT:\n{input_text}\n\nPlease produce the JSON as requested."
    try:
        rule_json = llm_json(llm, SYSTEM_SPLUNK_RULE, llm_input, max_retries=2)
    except Exception as e:
        raise RuntimeError("LLM không trả JSON rule hợp lệ: " + str(e))

    # 4) Basic validation / normalize fields
    # Ensure required keys exist and have defaults
    keys_defaults = {
        "title": "Generated Splunk Rule",
        "description": "",
        "spl": "",
        "condition_examples": [],
        "severity": "medium",
        "mitre": [],
        "schedule": "every 5m",
        "alert_actions": [],
        "tags": [],
        "detection_notes": ""
    }
    for k, dv in keys_defaults.items():
        if k not in rule_json:
            rule_json[k] = dv

    # optional: attach top evidence urls
    top_urls = []
    for e in evid[:6]:
        m = e.get("meta") or {}
        u = m.get("source_url") or m.get("source") or ""
        if u and u not in top_urls:
            top_urls.append(u)
    rule_json.setdefault("evidence_urls", top_urls)

    return rule_json

# --------------------- Example usage ---------------------
if __name__ == "__main__" and llm is not None:
    # Example: muốn rule phát hiện SQLi basic trên webapp
    try:
        inp = "Detect simple SQL injection attempts in web URI and POST params (payloads like \"' OR '1'='1\" , union select, sleep(5), benchmark)."
        print("\n🔧 Generating Splunk savedsearch for SQLi...")
        spl_rule = generate_splunk_rule(inp, alpha=0.55, topk=8)
        print("\n--- RULE JSON ---")
        print(json.dumps(spl_rule, ensure_ascii=False, indent=2))
    except Exception as e:
        print("⚠️ Lỗi khi sinh Splunk rule:", e)



🔧 Generating Splunk savedsearch for SQLi...


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"



--- RULE JSON ---
{
  "title": "Simple SQL Injection Detection",
  "description": "Detects simple SQL injection attempts in web URI and POST params. This rule looks for common SQL injection payloads like ' OR '1'='1, union select, sleep(5), and benchmark.",
  "spl": "search (index=web_logs \"' OR '1'='1\" OR \"union select\" OR \"sleep(5)\" OR \"benchmark\") | stats count as num_events by src_ip, uri_path, user_agent",
  "condition_examples": [
    "http://example.com/user?id=' OR '1'='1",
    "http://example.com/user?id=1 union select * from users"
  ],
  "severity": "high",
  "mitre": [
    "T1190"
  ],
  "schedule": "*/5 * * * *",
  "alert_actions": [
    "email",
    "notable",
    "webhook"
  ],
  "tags": [
    "web",
    "sqli",
    "input-validation"
  ],
  "detection_notes": "This rule may generate false positives if the application uses SQL-like syntax in its URI or POST params. Tuning suggestions: adjust the search query to exclude known false positives, and consider adding 